# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
is_colab = False

try:
    import google.colab
    is_colab = True
except ImportError:
    pass

In [2]:
if is_colab:
    %pip install sentence-transformers
    %pip install faiss-gpu-cu12

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
from tqdm.notebook import tqdm
import yfinance as yf
from openai import OpenAI

if is_colab:
    from google.colab import userdata
else:
    from dotenv import dotenv_values

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [4]:
K = 25

In [5]:
data_path = 'data/df_news.csv' if is_colab else '../data/df_news.csv'

In [6]:
df_news = pd.read_csv(data_path)
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [7]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [9]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [10]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [11]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [12]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [13]:
openai_key = None

if is_colab:
    openai_key = userdata.get('OPEN_AI_KEY')

else:
    config = dotenv_values('../.env')
    openai_key = config.get('OPEN_AI_KEY')

In [14]:
client = OpenAI(api_key=openai_key)

In [15]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

As the stars twinkled above, the gentle unicorn named Luna danced through a meadow of moonlit flowers, spreading dreams of magic and joy to every creature in the enchanted forest.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [16]:
import yfinance as yf

In [17]:
ticker_meta = {
    'TICKER': [],
    'COMPANY_NAME': [],
    'SECTOR': [],
    'INDUSTRY': []
}

for ticker in tqdm(df_news['TICKER'].unique()):
    yf_ticker = yf.Ticker(ticker)
    ticker_meta['TICKER'].append(ticker)
    ticker_meta['COMPANY_NAME'].append(yf_ticker.info['longName'])
    ticker_meta['SECTOR'].append(yf_ticker.info['sector'])
    ticker_meta['INDUSTRY'].append(yf_ticker.info['industry'])

  0%|          | 0/490 [00:00<?, ?it/s]

In [18]:
df_meta = pd.DataFrame(ticker_meta)
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [19]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [20]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [21]:
class RAGPipeline:
    def __init__(self, faiss_store, client):
        self.faiss_store = faiss_store
        self.client = client

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [doc for doc, _, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    @staticmethod
    def _generate_prompt_context(documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, _, _ in documents:
            context += f"- {doc}\n"

        return context

In [22]:
rag = RAGPipeline(faiss_store, client)

In [23]:
def display_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### Sources used for generating this answer:\n"

    docs_md = [f"- {doc}" for doc in docs]
    result += "\n".join(docs_md)

    print_markdown(result)

In [24]:
print_markdown("# Topic-focused questions")

for question in questions_topic:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news highlights several major concerns regarding inflation:

1. **Persistent Inflation**: The minutes from the Federal Reserve's May policy meeting indicate increasing worries about ongoing high inflation levels in the U.S., which could lead to a slowdown in economic growth.

2. **Impact on Monetary Policy**: Elevated food inflation complicates the Federal Reserve's strategy for potential rate cuts, suggesting that inflationary pressures are strong enough to influence monetary policy decisions in a way that could delay interest rate reductions.

3. **Economic Uncertainty**: The interplay of inflation with tariff issues and broader economic conditions has created uncertainty, raising fears about how these factors could affect consumer spending and overall economic stability.

Overall, the dominant themes revolve around the persistence of inflation, its implications for monetary policy, particularly interest rates, and the broader economic impacts driven by inflationary pressures.

### Sources used for generating this answer:
- Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines appears to be mixed, characterized by a combination of optimism and caution. Here are some key observations drawn from the articles:

1. **Optimism Around Certain Stocks**: There is a notable bullish sentiment surrounding several stocks, especially those that are outperforming the market. Headlines indicate that some stocks have caught Wall Street’s attention due to positive catalysts like new product launches or strong community support, suggesting a belief in their growth potential.

2. **Skepticism About Analyst Projections**: Despite the optimistic price targets set by analysts, there are clear warnings to remain skeptical. The articles highlight that analysts may face pressures that can lead to overly optimistic forecasts, often for their firms' business interests, such as M&A advisory. This creates a tension between positive sentiment in forecasts and the reality of underlying fundamentals.

3. **Concerns About Fundamentals**: Some articles caution investors about certain stocks that may have questionable fundamentals, despite the overwhelmingly bullish sentiment from Wall Street. This indicates that while the market may currently favor these stocks, there are significant concerns that could affect their long-term performance.

4. **Bearish Calls Stand Out**: When Wall Street analysts issue bearish ratings, it captures attention. This indicates a significant sentiment shift and suggests that investor caution could be warranted for specific stocks that analysts believe may not perform well.

In summary, while there is a prevailing wave of optimism about several stocks, particularly those with positive news and performance, there is also a strong undercurrent of skepticism regarding analyst forecasts and the fundamentals that support these investments. Investors are advised to exercise caution and critically assess the information presented in these optimistic headlines.

### Sources used for generating this answer:
- 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.
- 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.
- 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.
- 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.
- 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.

## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is playing a transformative role in the finance sector, as highlighted in several recent news stories. Here are the key trends and impacts:

1. **Enhanced Lending Technologies**: Jack Henry & Associates has integrated AI-driven lending technology, aiming to increase productivity and reduce human error in financial processes. This development underscores how AI is being utilized to streamline operations within traditional banking sectors.

2. **Investment Opportunities**: Companies like Meta Platforms are making significant investments in AI applications, which are beginning to influence their stock valuations. As tech giants pivot towards AI, investors are closely monitoring these developments to capture potential growth in their stock prices.

3. **Stock Market Movements**: AI-focused stocks such as Palantir and Upstart have garnered the attention of billionaire hedge fund managers, suggesting a rising confidence in AI's potential for significant returns. Upstart, for instance, uses AI to evaluate credit risk, offering a promising approach to lending that appeals to investors looking for sound valuations.

4. **Significant Gains**: Both Palantir and Snowflake have reported substantial stock gains despite broader market weaknesses, indicating that AI-related investments are gaining traction among investors. The successful application of generative AI in sectors like government and commercial operations points to its broad relevance.

5. **Content Production**: AI's ability to simplify and accelerate content production is also noted, as seen with Netflix's strategy. The company is leveraging AI to enhance its content offerings, which is crucial for maintaining its competitive edge in the media landscape.

Overall, AI is reshaping the finance sector by introducing innovative technologies, creating investment opportunities, and driving stock market performance, all while improving operational efficiency and risk assessment practices in lending and content creation.

### Sources used for generating this answer:
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation.
- Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

In [25]:
print_markdown("# Company-focused questions")

for question in questions_company:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
In recent news stories about artificial intelligence (AI), Microsoft is being portrayed as a significant player in the tech industry, alongside other major companies like Meta and Salesforce. The emphasis is on how Microsoft's strategic investments in AI could enhance its market position, drawing parallels with the "Magnificent Seven" stocks that are set to gain from AI advancements.

While specific articles may not focus exclusively on Microsoft, the company is implicitly included in discussions around market shifts and competitive dynamics spurred by AI. Analysts often highlight the challenges and strategies of other tech giants, indirectly reflecting on Microsoft's potential trajectory in the AI space. For instance, companies like Salesforce are noted for overcommitting to AI, which sets a contrasting backdrop to how Microsoft is positioning its AI strategy to capitalize on emerging opportunities.

Overall, Microsoft's portrayal leans towards being an innovator and strategic investor in AI, suggesting that if its investments yield strong results, it could significantly impact its stock and market performance in a rapidly evolving tech landscape.

### Sources used for generating this answer:
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here.
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.
- Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […]

## Q: What financial news headlines connect Amazon with automation or logistics?
Here are some recent financial news headlines connecting Amazon with automation or logistics:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article discusses how Amazon is leveraging artificial intelligence across all of its operations, which is closely tied to automation in logistics and supply chain management.

2. **Truist Reiterates Buy on Amazon.com as Q2 Revenue Tracks Ahead**: This piece indicates that Amazon's growth is possibly supported by innovative technologies, including automation, which may enhance its logistics and operational efficiency.

3. **Winners And Losers Of Q1: C.H. Robinson Worldwide Vs The Rest Of The Air Freight and Logistics Stocks**: While this article focuses on logistics stocks, it indirectly underscores the competitive landscape in which Amazon operates, especially in terms of logistics automation.

These articles highlight Amazon's efforts to integrate advanced technologies like AI and automation into its logistics and operational strategies.

### Sources used for generating this answer:
- Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]
- Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."
- Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi
- Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).
- Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).

In [26]:
print_markdown("# Industry-focused questions")

for question in questions_industry:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
The main themes emerging in financial news about the semiconductor industry, particularly regarding ON Semiconductor Corporation, include:

1. **Revenue Trends and International Exposure**: There is a significant focus on ON Semiconductor's international revenue trends. Investors and analysts are keen on understanding how these trends affect overall company performance and Wall Street forecasts.

2. **Investor Attention and Market Sentiment**: ON Semiconductor is attracting considerable interest from investors, indicating potential optimism about future performance despite some recent earnings concerns. This suggests a belief in the company's long-term prospects.

3. **Earnings Performance**: There is a discussion around ON Semiconductor's earnings, which have shown some softness. However, this has not deterred shareholder confidence, hinting at a loyal investor base that may be looking beyond short-term fluctuations.

4. **Spotting Investment Opportunities**: The articles also reflect a broader trend in the semiconductor industry where investors are seeking to identify promising stocks, highlighting companies like Vishay Intertechnology and Analog Semiconductors in the context of current market conditions.

5. **Influential Investors**: ON Semiconductor appears on lists associated with prominent investors, such as billionaire Glenn Russell Dubin, suggesting that the company is on the radar of influential financial figures, which could be indicative of its potential growth and strategic importance in the industry. 

Overall, the semiconductor industry is characterized by fluctuating earnings, investor interest, and international revenue considerations, all indicating a dynamic investment landscape.

### Sources used for generating this answer:
- Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.
- ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.
- Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...
- Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.
- ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]

## Q: What trends are being reported in the retail industry?
The retail industry is currently experiencing several significant trends shaped by various external factors:

1. **Shift in Business Models**: Retailers are adapting their strategies in response to technological advancements that change consumer shopping habits. This includes increasing reliance on e-commerce and integrating technology into physical retail spaces.

2. **Volatile Demand**: The retail sector is heavily influenced by consumer spending, which can fluctuate greatly. Recent reports indicate a decline in retail stocks, with a 13.7% drop over the past six months, reflecting the industry's exposure to economic cycles.

3. **Impact of Trade Policies**: Retailers are navigating challenges posed by tariffs and trade policies, especially those implemented during the previous administration. Many have strategically altered their supply chains and adjusted pricing to mitigate impacts from these changes.

4. **Economic Sensitivity**: The performance of consumer discretionary businesses, which includes many retailers, tends to mirror broader economic trends. The sector has seen a significant downturn, dropping by 12.3%—substantially more than the S&P 500's decline of 2.1%, indicating a tough environment for retail stocks.

These trends suggest that while the retail industry is making strides towards modernization, it faces considerable challenges that affect its stability and growth prospects.

### Sources used for generating this answer:
- 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.
- Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.
- 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.
- Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...
- Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news about the energy industry highlights several significant risks and challenges:

1. **Legislative Risks**: The passage of a bill in the U.S. House of Representatives that aims to repeal key subsidies for renewable energy presents a substantial threat to the industry's economic viability. If enacted, this legislation could render many existing and future renewable projects uneconomical, leading to a crash in renewable energy stocks.

2. **Volatility in Oil and Gas Prices**: The oilfield service sector faces challenges due to sliding oil prices. This volatility can lead to reduced investment and budget cuts for drilling, which subsequently affects profits and operational stability for companies like SLB, HAL, and BKR.

3. **Tariffs and Trade Wars**: The trade conflicts, particularly related to tariffs, pose additional risks. Rising tariffs can increase operational costs for energy companies and complicate international supply chains.

4. **Economic Pressures on Investment**: With increasing tariffs and the uncertain legislative environment, investor confidence may wane, further tightening the budgets for energy projects and exploration.

5. **Dependency on Emerging Technologies**: While there may be opportunities related to LNG (liquefied natural gas) and AI (artificial intelligence), the industry must navigate a challenging landscape to leverage these technologies effectively to sustain demand and profitability.

Overall, the combination of regulatory challenges, market fluctuations, and geopolitical factors creates a complex environment for the energy industry that may impact its growth and sustainability in the near future.

### Sources used for generating this answer:
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

For the most part, the answers are generated from the retrieved articles. All of the key points mentioned by the LLM can be traced to a specific article. However, there are cases in which the LLM does introduce more ideas. For example, in the "What financial news headlines connect Amazon with automation or logistics?" question, one of the selected headlines ("Winners And Losers Of Q1: C.H. Robinson Worldwide Vs The Rest Of The Air Freight and Logistics Stocks") is solely about logistic companies; the LLM however, made the connection between it and the "competitive landscape in which Amazon operates, especially in terms of logistics automation."

While this is arguably more of a fault with the document retrieval process (the vector store most likely returned that specific article because of the "automation or logistics" part of the query), it is interesting to see how the LLM tries to "connect the dots" between the query and each of the documents provided; even if in reality, there is no connection. This indicates that the LLM will try to always use all the retrieved documents as part of the generation, even when it may not be the appropriate thing to do.

#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

It does address the question. However, in some cases, there may be more context required to generate a more thorough response. For example, in the "How is Microsoft being portrayed in news stories about artificial intelligence?" question, the answer states that Microsoft is not directly mentioned in the articles. While Microsoft specifically is not mentioned, it is a part of what's known as the "Magnificent Seven", which are mentioned and the LLM even references in its answer by saying the articles are "drawing parallels" between Microsoft and the Magnificent Seven.

While the articles may indeed be relevant, they might be so for reasons that are not clear to the LLM based solely on the provided context, which may limit the capability for more in-depth answers.

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

Yes, there are irrelevant news snippets that are being added to the context, like the example mentioned for the Amazon question which retrieved a news article about the logistics industry instead. In the Microsoft question there are also articles about other companies, like Salesforce, Meta, Jack Henry, and Netflix; which appear to be retrieved only because they discuss AI.

As mentioned, the main impact of this is that the LLM will try to include the provided context as a truth in its answer, so the output will be heavily influenced to account for all of the provided articles. In the cases mentioned (Amazon and Microsoft), the LLM tried to relate the articles to each company by connecting them to the industry they operate in, even if the article is not discussing the company.

#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

For the most part, the news snippets are consistent. However, there are cases like the answer for the "How is investor sentiment described in recent financial headlines?" question, that may have contradictory articles used for LLM context. In cases like this, the LLM does try to account for all answers by treating all stances as equally valuable. For example, for that question, some of the key observations are that there is "optimism around *certain* stocks"; however, there is "skepticism about analyst projections." As can be seen, the LLM tries to have a neutral viewpoint, even if it means treating opposite stances as equally valid.

This is potentially because of the specific prompt used, which made clear that the context comes from news articles, which may cause the LLM to consider the inherent bias in news outlets.

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

The main improvement I can think of is to prevent duplicate retrieved documents. This happens because there are articles that are related to multiple companies, so they appear more than once in the dataset with the only difference being the specific ticker it is related to. A potential way to solve this would be to normalize the data, so that each article only appears once. To preserve the many-to-many relationship between articles and companies, we could have a secondary auxiliar table or dataframe connecting companies to articles; and instead of listing a single ticker in the documents metadata when building the vector store, we can have it as a set of tickers related to the company. While this would require more logic to be applied to the vector store query implementation, it would avoid duplicate embeddings from being stored, and as such, introduce more variety to the retrieved documents used for LLM context.

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [27]:
class EnrichedRAGPipeline:
    def __init__(self, faiss_store, client, company_metadata):
        self.faiss_store = faiss_store
        self.client = client
        self.company_metadata = company_metadata

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [(doc, self._get_company_metadata(meta['TICKER'])) for doc, meta, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.
        This added context will take the form of recent news article related to
        the question. Each of these articles will be about a specific
        company, sector or industry.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    def _generate_prompt_context(self, documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, meta, _ in documents:
            ticker = meta['TICKER']
            c_meta = self._get_company_metadata(ticker)
            context += f"This following news article is related to the '{c_meta['COMPANY_NAME']}' company ({ticker}), operating in the {c_meta['SECTOR']} sector and the {c_meta['INDUSTRY']} industry."
            context += f"> {doc}\n"

        return context

    def _get_company_metadata(self, ticker):
        return self.company_metadata[self.company_metadata['TICKER'] == ticker].to_dict(orient='records')[0]

In [28]:
enriched_rag = EnrichedRAGPipeline(faiss_store, client, df_meta)

In [29]:
def display_enriched_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### The following news articles were used for generating this answer:\n"
    result += "| Ticker | Company | Sector | Industry | Article |\n"
    result += "|--------|--------------|--------|----------|-----|\n"
    for doc, meta in documents:
        result += f"| {meta['TICKER']} | {meta['COMPANY_NAME']} | {meta['SECTOR']} | {meta['INDUSTRY']} | {doc} |\n"
    print_markdown(result)

In [30]:
print_markdown("# Topic-focused questions")
for question in questions_topic:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news has highlighted several major concerns regarding inflation:

1. **Persistent Inflation Risks**: Articles indicate that there is a growing concern among Federal Reserve officials about the persistence of U.S. inflation. Minutes from the Fed's May meeting revealed worries that inflation may not be declining as hoped and that it could lead to economic slowdown.

2. **Impact on Monetary Policy**: There's a significant connection between inflation and the Fed’s monetary policy decisions. The articles imply that continued inflation, particularly in food prices, dampens expectations for a potential interest rate cut. This creates uncertainty for investors and consumers alike.

3. **Sector-Specific Vulnerabilities**: Different sectors are feeling the impact of inflation. For example, companies in the consumer cyclical sector, such as Tesla and lululemon, are facing challenges due to rising prices, which may impact consumer spending behavior.

4. **Tariff Twists**: The ongoing complexities related to tariffs further complicate the inflation landscape. As tariffs can lead to increased costs for imported goods, the potential for sustained inflation increases, impacting various industries.

Overall, concerns revolve around the persistence of inflation, its impact on monetary policy, sector-specific vulnerabilities, and external factors like tariffs that exacerbate the inflation problem.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| BLK | BlackRock, Inc. | Financial Services | Asset Management | Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown. |
| TSLA | Tesla, Inc. | Consumer Cyclical | Auto Manufacturers | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| NVDA | NVIDIA Corporation | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| LULU | lululemon athletica inc. | Consumer Cyclical | Apparel Retail | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| AVGO | Broadcom Inc. | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |


## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines appears to be a mix of optimism and skepticism, with notable emphasis on bullish outlooks alongside caution regarding the reliability of analysts' forecasts.

1. **Optimism in Certain Stocks**: Articles related to companies like Microchip Technology and Monolithic Power Systems highlight strong recent performances and positive catalysts, suggesting that these stocks are favored by investors and experiencing momentum. Price targets exceeding 20% returns indicate a bullish sentiment, particularly as these companies are associated with growth sectors like technology and semiconductors.

2. **Caution Regarding Analyst Forecasts**: Despite the optimism, there is a clear thread of skepticism. Articles indicate that analysts may face institutional pressures leading to overly optimistic predictions. For example, the discussion around CarMax and Darden Restaurants mentions that bearish sentiments should not be overlooked, as analysts hesitate to issue negative ratings due to fear of losing future business, thereby questioning the reliability of positive forecasts.

3. **Mixed Sentiments in Consumer Cyclical and Healthcare**: In sectors like Consumer Cyclical and Healthcare, articles describe a polarized view where certain stocks are highlighted as undervalued or overlooked, while others face bearish sentiments. This leads to a nuanced perspective, where some investors might feel confident in specific picks, while others remain cautious or skeptical of overall market conditions and the reliability of analyst predictions.

In summary, investor sentiment is characterized by a blend of enthusiasm for certain stocks, tempered by a cautious understanding of the motives and pressures that may influence analyst ratings.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts. |
| MCHP | Microchip Technology Incorporated | Technology | Semiconductors | 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase. |
| MPWR | Monolithic Power Systems, Inc. | Technology | Semiconductors | 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory. |
| DRI | Darden Restaurants, Inc. | Consumer Cyclical | Restaurants | 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory. |
| RVTY | Revvity, Inc. | Healthcare | Diagnostics & Research | 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover. |


## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is playing a transformative role across various sectors in finance and technology, as highlighted by recent news articles involving several key companies. Here's an overview of its significance:

1. **Productivity and Efficiency**: Jack Henry & Associates is integrating AI-driven lending technology to enhance productivity and reduce human error. This signifies a broader trend where financial services leverage AI to streamline operations and improve customer service.

2. **Valuation and Investment Strategies**: Meta Platforms has heavily invested in various AI applications, with expectations that these investments will significantly impact its future growth. This has implications for how investors assess tech companies, often valuing them based on potential AI-driven innovations rather than just their current business models.

3. **Stock Performance and AI Demand**: Companies like Palantir are successfully tapping into the demand for AI solutions, particularly in government and commercial sectors. Their ability to integrate generative AI capabilities into client operations is raising investor interest and influencing stock valuations. Analysts predict substantial upside for AI-related stocks, affecting investment strategies.

4. **Content Production**: Netflix is exploring generative AI to enhance its content production process. This has implications not only for its operational efficiency but also for its competitive positioning in the crowded entertainment market.

In summary, AI is increasingly viewed as a critical driver of innovation and profitability in the finance and tech sectors, influencing investment decisions, operational efficiencies, and overall company valuations.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago. |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |


In [31]:
print_markdown("# Company-focused questions")
for question in questions_company:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
The provided context does not include any specific news articles about Microsoft directly; rather, it focuses on other companies like Meta Platforms, Salesforce, Jack Henry & Associates, and Netflix. Therefore, I’ll summarize how Microsoft is portrayed in the broader context of artificial intelligence based on general industry trends and insights.

Microsoft is often depicted in a positive light regarding its advancements in artificial intelligence. Key highlights in recent narratives about Microsoft’s AI efforts include:

1. **Investments in AI**: Microsoft has been making significant investments in AI technology, especially through partnerships with companies like OpenAI. Their integration of AI into products like Microsoft Office and Azure is frequently highlighted as a transformative effort for their software ecosystem.

2. **Market Leadership**: The company is viewed as a strong competitor in the AI space, often compared favorably against peers such as Google and Amazon. Microsoft’s commitment to AI innovations boosts its reputation as a leader in this technology sector.

3. **Focus on Practical Applications**: Microsoft emphasizes developing AI solutions that solve real-world problems, which is reflected in their marketing and product development strategies. This practical approach is generally well-received, as it showcases the utility of their AI offerings.

4. **Ethical AI Development**: Microsoft often discusses its commitment to ethical AI development, which resonates positively with stakeholders concerned about the implications of AI technology.

In summary, while the context does not provide specific reports on Microsoft, the general narrative surrounding the company in relation to AI is positive, emphasizing their strategic investments, market leadership, practical applications, and ethical considerations in AI development.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| CRM | Salesforce, Inc. | Technology | Software - Application | How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here. |
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […] |


## Q: What financial news headlines connect Amazon with automation or logistics?
Here are some financial news headlines that connect Amazon with automation or logistics:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article discusses how Amazon is leveraging artificial intelligence across its operations, which plays a crucial role in enhancing its logistics and automation capabilities.

2. **Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead**: Although this article is primarily about Amazon's financial performance, it aligns with the broader conversation around how technology and automation can drive revenue, particularly in e-commerce.

These articles highlight Amazon's ongoing efforts in automation and the integration of AI into its business model, particularly in logistics and supply chain management.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […] |
| AMZN | Amazon.com, Inc. | Consumer Cyclical | Internet Retail | Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology." |
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi |
| AON | Aon plc | Financial Services | Insurance Brokers | Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS). |
| CHRW | C.H. Robinson Worldwide, Inc. | Industrials | Integrated Freight & Logistics | Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW). |


In [32]:
print_markdown("# Industry-focused questions")
for question in questions_industry:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
In recent financial news about the semiconductor industry, several key themes have emerged:

1. **International Revenue Trends**: There's a growing focus on how international sales are shaping forecasts and expectations for companies like ON Semiconductor. The impact of global market dynamics on revenue generation is becoming crucial for investors.

2. **Investor Sentiment and Attention**: Companies such as ON Semiconductor are attracting significant interest from investors. The stock's performance and investor sentiment, even amid fluctuating earnings reports, suggest a strong belief in their long-term growth potential.

3. **Earnings Performance**: Despite soft earnings in certain instances, such as those reported by ON Semiconductor, there appears to be a resilience among shareholders. This could indicate optimism in the company's future prospects despite short-term challenges.

4. **Market Competition and Innovation**: Articles hint at an ongoing assessment of market competitors like Vishay Intertechnology and their evolving roles within the analog semiconductors segment, demonstrating the competitiveness of the industry.

5. **Stock Picks by Influential Investors**: The interest from prominent investors, such as billionaire Glenn Russell Dubin, in stocks like ON Semiconductor underscores the belief in substantial upside potential in the semiconductor sector, hinting at a strategic move towards growth-oriented semiconductor stocks.

These themes collectively highlight a sector characterized by careful monitoring of revenue trends, investor confidence, and competitive dynamics, alongside the influence of key market players.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week... |
| ADI | Analog Devices, Inc. | Technology | Semiconductors | Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […] |


## Q: What trends are being reported in the retail industry?
Recent trends in the retail industry reflect a challenging environment influenced by several factors:

1. **Technological Adaptation**: Retailers are evolving their business models in response to technological advances that are changing consumer shopping behaviors. Companies are integrating online and offline experiences to accommodate the growing preference for digital shopping.

2. **Volatility in Consumer Demand**: The retail sector has experienced significant fluctuations in consumer spending, contributing to a volatile market. Recent statistics indicate a notable decline in retail stocks, which have fallen by 13.7% over the past six months, surpassing the S&P 500's decline of 5.5%. This suggests that consumer discretionary spending may be weakening.

3. **Response to Economic Cycles**: The performance of retail businesses is closely tied to broader economic trends. The recent drop of 12.3% in the consumer discretionary sector over the past six months, compared to a 2.1% drop in the S&P 500, highlights how sensitive the sector is to macroeconomic changes.

4. **Supply Chain Adjustments**: Retailers are modifying their supply chains in light of trade policies and tariffs. Executives are adjusting pricing strategies, with some price increases already realized in the market. This ongoing adjustment reflects a proactive approach to navigating economic uncertainties.

Overall, the retail industry is grappling with the dual challenges of adapting to changing consumer behaviors while managing the impacts of economic fluctuations and trade policies.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss. |
| BBY | Best Buy Co., Inc. | Consumer Cyclical | Specialty Retail | Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves. |
| HLT | Hilton Worldwide Holdings Inc. | Consumer Cyclical | Lodging | 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline. |
| PKG | Packaging Corporation of America | Consumer Cyclical | Packaging & Containers | Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst... |
| APD | Air Products and Chemicals, Inc. | Basic Materials | Specialty Chemicals | Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to... |


## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news highlights several significant risks and challenges facing the energy industry:

1. **Legislative Risks for Renewable Energy**: A major challenge is the potential repeal of subsidies vital to the renewable energy sector. A bill advancing through Congress threatens to undermine economic viability for renewable energy projects. This could result in significant financial setbacks for companies like NextEra Energy and Enphase Energy, potentially leading to declines in stock performance and investment.

2. **Volatile Oil Prices**: The oil and gas sector faces downward pressure on profitability as oil prices continue to slide. Companies in this space, such as Halliburton and Baker Hughes, are grappling with shrinking drilling budgets, which may limit project initiation and expansion. This market uncertainty could lead to decreased revenues and profitability.

3. **Tariffs and Trade Policies**: Rising tariffs complicate the operational landscape for energy companies, particularly those involved in oilfield services. These tariffs could increase costs, squeeze margins, and impact competitiveness in the global market.

4. **Budget Constraints**: As oil prices decrease, companies are likely to face tighter budgets, which can hinder operational capabilities and innovation in the energy sector. The question remains whether demand for liquefied natural gas (LNG) and advancements in artificial intelligence (AI) can offset these challenges.

These factors collectively pose a significant risk to the stability and growth of the energy industry as firms navigate legislative changes, commodity price fluctuations, trade policy impacts, and budget constraints.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| NEE | NextEra Energy, Inc. | Utilities | Utilities - Regulated Electric | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| ENPH | Enphase Energy, Inc. | Technology | Solar | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| HAL | Halliburton Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| BKR | Baker Hughes Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| FCX | Freeport-McMoRan Inc. | Basic Materials | Copper | 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty. |


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE